In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import csv
import janitor
import numpy as np

In [2]:
# sample list
mapping_16S = pd.read_csv("~/Documents/thesis/16S/qiime2/mappingfiles/16S_mapping.csv")
mapping_16S.head()

,#SampleID,Mother_Child,SubjectID,MaternalID,TimePoint,Mgx_batch Mapping,16S_batch Mapping
0,C0052-5F-1A,C,52,0052_m,5,Mgx_batch001,16S_batch001
1,C0126-8F-1A,C,126,0126_m,8,Mgx_batch001,16S_batch001
2,C0126-8F-1B,C,126,0126_m,8,Mgx_batch001,16S_batch001
3,C0127-6F-1B,C,127,0127_m,6,Mgx_batch001,16S_batch001
4,C0286-5F-1A,C,286,0286_m,5,Mgx_batch001,16S_batch001


In [3]:
samplenames = list(mapping_16S["#SampleID"])

In [4]:
len(samplenames)

345

In [5]:
# read in mgx data
mgx = pd.read_csv("mgx_genera.csv")
mgx.rename(columns={'Taxon':'taxa'}, inplace=True)
mgx.head()


,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
0,Acidaminococcaceae_unclassified,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
1,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.002632,0.0,0.0,0.000000,0.03501,0.0,0.0
2,Acinetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
3,Actinobacillus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000139,0.000000,0.0,0.0,0.000116,0.00000,0.0,0.0
4,Actinobaculum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0


In [6]:
# ratio of unclassified organisms profiled with mgx
sum(mgx.taxa.str.contains("_unclassified| noname"))/mgx.shape[0]

0.041916167664670656

In [7]:
# remove taxa that are unclassified or have no name
# "_unclassified"
# "_noname"
mgx = mgx[~mgx.taxa.str.contains("_unclassified")]
mgx = mgx[~mgx.taxa.str.contains("_noname")]

In [8]:
mgx.sample(15)

,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
96,Megamonas,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.320095,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
73,Flavonifractor,0.000000,0.000050,0.000191,0.000000,0.000363,0.000165,0.000000,0.000000,0.000020,...,0.000000,0.000000,0.000000,0.000307,0.000967,0.000000,0.000000,0.001033,0.000000,0.000000
128,Prevotella,0.292187,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001112,0.000000,0.000000
20,Bacteroides,0.030745,0.332798,0.327179,0.200086,0.273605,0.196060,0.336870,0.156865,0.437279,...,0.277648,0.000000,0.273429,0.241254,0.314770,0.622911,0.405827,0.466731,0.209016,0.196054
10,Alistipes,0.024938,0.032657,0.034518,0.039037,0.012053,0.000000,0.000207,0.065534,0.000000,...,0.069910,0.000000,0.001220,0.097019,0.089590,0.083275,0.049778,0.247798,0.140833,0.167917
31,Butyricimonas,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
53,Deinococcus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
113,Pantoea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
85,Jonquetella,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Actinobacillus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000139,0.000000,0.000000,0.000000,0.000116,0.000000,0.000000,0.000000


In [9]:
mgx.to_csv('mgx_abundance.csv', index=False)

In [10]:
# read in 16S data
amplicon = pd.read_csv("feature-table.csv")
amplicon.rename(columns={'#OTU ID':'taxa'}, inplace=True)


In [11]:
# clean taxonomic names
# keep only genera
amplicon["taxa"] = amplicon['taxa'].str.split("D_5__", expand=True)[1]
amplicon.dropna(inplace=True)


In [12]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
0,Methanobrevibacter,0,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,19,0,0,0
7,Subgroup 10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Subgroup 23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Ilumatobacter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Actinomyces,0,0,7,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
len(amplicon.columns.values)

264

In [14]:
amplicon.sort_values(by = "taxa", ascending=True, inplace=True)

In [15]:
amplicon["taxa"] = amplicon["taxa"].astype(str)

In [16]:
# ratio of unclassified genera for 16S profiling

sum(amplicon.taxa.str.contains("_unclassified| unidentified| uncultured| group"))/amplicon.shape[0]

0.06435643564356436

In [17]:
# calculate relative abundances
bug_sum = amplicon.sum(axis = 0, skipna = True)
amplicon.iloc[:, 1:] = amplicon.iloc[:, 1:]/bug_sum

In [18]:
# remove any Nones, uncultured organisms
amplicon = amplicon[~amplicon.taxa.str.contains("uncultured")]
amplicon = amplicon[~amplicon.taxa.str.contains("metagenome")]
amplicon = amplicon[~amplicon.taxa.str.contains("unidentified")]
amplicon = amplicon[~amplicon.taxa.str.contains(r'[0-9]')]
amplicon = amplicon[~amplicon.taxa.str.contains("group")]

In [19]:
amplicon.to_csv('16S_abundance.csv')

### looking for the intersection of taxonomy

In [20]:
# calculate number of taxa in both vs. either one

In [21]:
amplicon_taxa_list = set(amplicon["taxa"])
mgx_taxa_list = set(mgx["taxa"])

In [22]:
intersection = amplicon_taxa_list.intersection(mgx_taxa_list)

union = amplicon_taxa_list.union(mgx_taxa_list)

In [23]:
union_df = pd.DataFrame(union)
union_df.to_csv('taxalist.csv', index=False)

In [24]:
len(intersection)
len(amplicon_taxa_list)-len(intersection)
len(mgx_taxa_list)-len(intersection)

84

159

59

In [25]:
amplicon_only = amplicon_taxa_list.difference(intersection)
mgx_only = mgx_taxa_list.difference(intersection)


In [26]:
len(amplicon_only)
len(mgx_only)

159

59

### creating long-form dataframe with data from both sequencing methods

In [27]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
245,Acetanaerobacterium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
335,Acetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438,Acholeplasma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,Achromobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.010618,0.008632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# only keeping samples we have both mgx and 16S profiles for
samples_intersect = set(mgx.columns.values).intersection(set(amplicon.columns.values))

In [29]:
# remove mothers
children_intersect = [id for id in samples_intersect if not id.startswith("M")]

In [30]:
# mgx[(samples_intersect, axis=1, inplace=True)
# amplicon.drop(samples_intersect, axis=1, inplace=True)

In [31]:
mgx = mgx[children_intersect]
amplicon = amplicon[children_intersect]

In [32]:
# confirm that we now have the same samples
(amplicon.columns.values) == (mgx.columns.values)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [33]:
amplicon_melt = pd.melt(amplicon, id_vars=["taxa"], var_name = "sampleid", value_name = "amplicon_abund")

In [34]:
mgx_melt = pd.melt(mgx, id_vars=["taxa"], var_name = "sampleid", value_name = "mgx_abund")

In [35]:
merged_taxa = pd.merge(amplicon_melt, mgx_melt, on = ["sampleid", "taxa"], how = "outer")

In [36]:
merged_taxa.to_csv('taxa_abundance_comparison.csv')

In [37]:
merged_taxa["abs_diff"] = abs(merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])
merged_taxa["tot_diff"] = (merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])

In [38]:
merged_taxa.sample(10)

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
13082,Rubricoccus,C0725-1E-1A,0.00000,NaN,NaN,NaN
22964,Intestinibacter,C1009-1F-1A,0.00076,NaN,NaN,NaN
13924,Dokdonia,C0672-1E-1A,0.00000,NaN,NaN,NaN
1939,Virgulinella fragilis,C1102-1F-1A,0.00000,NaN,NaN,NaN
16112,Dorea,C0739-2F-1A,0.00000,0.0,0.0,0.0
12246,Flavitalea,C1026-1F-1A,0.00000,NaN,NaN,NaN
21991,Ilumatobacter,C0834-4F-1A,0.00000,NaN,NaN,NaN
28483,Alloscardovia,C0661-2F-1A,NaN,0.0,NaN,NaN
3818,Paenibacillus,C0107-4E-1A,0.00000,NaN,NaN,NaN
16928,Negativibacillus,C0698-2F-1A,0.00000,NaN,NaN,NaN


In [39]:
merged_taxa.fillna(0)

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
0,Acetanaerobacterium,C0469-2E-1A,0.0,0.0,0.0,0.0
1,Acetobacter,C0469-2E-1A,0.0,0.0,0.0,0.0
2,Acholeplasma,C0469-2E-1A,0.0,0.0,0.0,0.0
3,Achromobacter,C0469-2E-1A,0.0,0.0,0.0,0.0
4,Acidaminococcus,C0469-2E-1A,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
29893,Tannerella,C2018-4F-1A,0.0,0.0,0.0,0.0
29894,Treponema,C2018-4F-1A,0.0,0.0,0.0,0.0
29895,Tunalikevirus,C2018-4F-1A,0.0,0.0,0.0,0.0
29896,Ureaplasma,C2018-4F-1A,0.0,0.0,0.0,0.0


In [40]:
amplicon_avg_abund = merged_taxa.groupby("taxa")["amplicon_abund"].mean()
mgx_avg_abund = merged_taxa.groupby("taxa")["mgx_abund"].mean()
taxa_list = sorted(set(merged_taxa["taxa"]))

In [41]:
mean_taxa_abund = pd.DataFrame(
    (zip(taxa_list, amplicon_avg_abund, mgx_avg_abund)),  
    columns = ['taxa','amp_avg_abund', 'mgx_avg_abund'])

In [42]:
mean_taxa_abund["abs_diff"] = abs(mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"])
mean_taxa_abund["total_diff"] = mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"]

In [43]:
mean_taxa_abund.sort_values("abs_diff", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [44]:
mean_taxa_abund.sample(10)

,taxa,amp_avg_abund,mgx_avg_abund,abs_diff,total_diff
80,Desulfatiglans,0.000032,NaN,NaN,NaN
141,Harryflintia,0.000000,NaN,NaN,NaN
231,Porphyromonas,0.000112,0.000063,0.000049,0.000049
22,Amylibacter,0.000017,NaN,NaN,NaN
273,Stenotrophomonas,0.000021,0.000002,0.000019,0.000019
35,Aurantivirga,0.000018,NaN,NaN,NaN
279,Sulfitobacter,0.000240,NaN,NaN,NaN
102,Enterorhabdus,0.000000,NaN,NaN,NaN
152,Intestinimonas,0.000482,NaN,NaN,NaN
85,Desulfosarcina,0.000023,NaN,NaN,NaN


In [45]:
mean_taxa_abund.to_csv('taxa_difference.csv')

## making giant dataframe of abundances

In [46]:
amp_trans = amplicon.set_index("taxa").transpose()

In [47]:
amp_trans.reset_index(level=0, inplace=True)

In [48]:
amp_trans.rename(columns = {'index':'sampleid'}, inplace = True) 

In [49]:
amp_trans["uid"] = amp_trans["sampleid"].astype(str)+'-amp'

In [50]:
amp_trans["method"] = "amp"

In [51]:
amp_trans.head()

taxa,sampleid,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,Actibacter,Actinobacillus,Actinomyces,...,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts,uid,method
0,C0469-2E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0469-2E-1A-amp,amp
1,C0654-2F-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.002268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0654-2F-1A-amp,amp
2,C0123-3E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006231,...,0.017134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0123-3E-1A-amp,amp
3,C0694-1E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0694-1E-1A-amp,amp
4,C0402-3F-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0402-3F-1A-amp,amp


In [52]:
mgx_trans = mgx.set_index("taxa").transpose()

In [53]:
mgx_trans.reset_index(level=0, inplace=True)

In [54]:
mgx_trans.rename(columns = {'index':'sampleid'}, inplace = True)

In [55]:
mgx_trans["uid"] = mgx_trans["sampleid"].astype(str)+'-mgx'

In [56]:
mgx_trans["method"] = "mgx"

In [57]:
mgx_trans.head()

taxa,sampleid,Acidaminococcus,Acinetobacter,Actinobacillus,Actinobaculum,Actinomyces,Adlercreutzia,Aeromonas,Aggregatibacter,Akkermansia,...,Tannerella,Treponema,Tunalikevirus,Turicibacter,Ureaplasma,Varibaculum,Veillonella,Weissella,uid,method
0,C0469-2E-1A,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000264,0.0,C0469-2E-1A-mgx,mgx
1,C0654-2F-1A,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.005967,0.0,C0654-2F-1A-mgx,mgx
2,C0123-3E-1A,0.0,0.0,0.0,0.0,0.0,0.000331,0.0,0.0,0.001331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000348,0.0,C0123-3E-1A-mgx,mgx
3,C0694-1E-1A,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.001948,...,0.0,0.0,0.0,0.0,0.0,0.0,0.001320,0.0,C0694-1E-1A-mgx,mgx
4,C0402-3F-1A,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,C0402-3F-1A-mgx,mgx


In [58]:
concat_df = pd.concat([mgx_trans,amp_trans], sort=True).reset_index(drop = True)

In [59]:
len(concat_df)

198

In [60]:
age = pd.read_csv("~/Documents/thesis/analysis/metadatawide.csv")

/Users/danielle/miniconda2/envs/thesis_analysis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (22,23,99,110,152,212,252,291,294,321,326,415,440,540,547,574,577) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
age = age[['sample','childAgeMonths']]

In [62]:
age["sample"] = age["sample"].str.replace("_",'-')
age.rename(columns = {'sample':'sampleid'}, inplace = True) 

In [63]:
# make age dictionary
agedict = {str(s): {} for s in age["sampleid"]}
for index, row in age.iterrows():
    age_months = row["childAgeMonths"]
    agedict[row["sampleid"]]= age_months

In [64]:
concat_df["AgeMonths"]= concat_df["sampleid"].map(agedict)

In [65]:
cols_to_order = ['uid', 'sampleid',"method", "AgeMonths"]
new_columns = cols_to_order + (concat_df.columns.drop(cols_to_order).tolist())
concat_df = concat_df[new_columns]

In [66]:
concat_df.sample(15)

,uid,sampleid,method,AgeMonths,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,...,Ureaplasma,Varibaculum,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts
64,C0286-6E-1A-mgx,C0286-6E-1A,mgx,91.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.000835,NaN,NaN,NaN,0.0,NaN,NaN,NaN
93,C2014-3F-1A-mgx,C2014-3F-1A,mgx,25.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.008741,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,C0402-3F-1A-mgx,C0402-3F-1A,mgx,77.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN
131,C0754-3F-1A-amp,C0754-3F-1A,amp,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.000898,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
166,C0752-4F-1A-amp,C0752-4F-1A,amp,15.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.235770,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
132,C0052-7E-1A-amp,C0052-7E-1A,amp,143.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.000606,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
7,C1102-1F-1A-mgx,C1102-1F-1A,mgx,92.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN
76,C0683-1E-1A-mgx,C0683-1E-1A,mgx,53.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.000372,NaN,NaN,NaN,0.0,NaN,NaN,NaN
89,C0713-4F-1A-mgx,C0713-4F-1A,mgx,16.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.015230,NaN,NaN,NaN,0.0,NaN,NaN,NaN
145,C0788-2F-1A-amp,C0788-2F-1A,amp,109.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.000468,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [67]:
concat_df.to_csv('transposed_mgxamp_df.csv')

In [68]:
pwd()

'/Users/danielle/Documents/thesis/analysis'